## Build a Delft3D FM 1D2D model from scratch

This notebook demonstrates how to prepare a **1D2D Delft3D FM** model from scratch using the command line interace (CLI). 

We will combine elements that were prepared in the *Build 1D model* and *Build 2D model* notebooks, so we advise you to have a look at these before starting this one.

All lines in this notebook which start with `!` are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste these in your shell to get more direct feedback.

### HydroMT CLI build interface

Lets first check if the Delft3D FM model is recognized by HydroMT:

In [ ]:
# this should return "dflowfm" to build DFlowFM components of Delft3D FM
# as well as the generic HydroMT models "grid_model, lumped_model, mesh_model, network_model"
!hydromt --models

Using the **HydroMT build** API we can setup a complete model from scratch. Let's get an overview of all the available options:

In [ ]:
!hydromt build --help

### Data for model setup

Most of the time, the `<...>_fn` arguments correspond to a data source from the `DataCatalog` which is based on a yml-file with references to the data paths/urls and how the data should be read. This file can be provided to hydromt using `-d <path_to_yml_file>`. 

By default, the pre-defined `artifact_data` catalog containing some example data for the Piave basin will be downloaded to `~/.hydromt_data/` which is also used for this example. An overview of the available example data is provided [here](https://deltares.github.io/hydromt/latest/user_guide/data_existing_cat.html).
This example data is a based on the data which is available from the Deltares p-drive. 
If you have acces to this drive, a pre-configured catalog file can be loaded using the `-d deltares_data` pre-defined catalog. 

More information on how to write a data catalog yml file can be found in the [HydroMT core docs](https://deltares.github.io/hydromt/latest/user_guide/data_prepare_cat.html) 

> **NOTE**: In HydroMT-Delft3D FM, an additional data catalog with all default parameters values is always used [parameters_data.yml](https://github.com/Deltares/hydromt_delft3dfm/tree/main/hydromt_delft3dfm/data/parameters_data.yml). It contains default values for 1D network elements and structures as well as default 2D landuse parameter mapping tables.

### Model setup configuration

The HydroMT configuration file contains the model setup configuration and determines which methods are used to prepare the different components of a Delft3D FM model and in which order and optionally sets non-default arguments for each method. This configuration is passed to HydroMT using `-i <path_to_configuration_file>`. We have prepared several example yml-files which are available in the model repository [examples folder](https://github.com/Deltares/hydromt_delft3dfm/tree/main/examples) and from the [docs (building a model)](https://deltares.github.io/hydromt_delft3dfm/latest/user_guide/dflowfm_build). 

Each section, before indent, (e.g. setup_rivers_from_dem) corresponds to a model method. All model methods are explained in the [docs (model components)](https://deltares.github.io/hydromt_delft3dfm/latest/user_guide/dflowfm_model_setup.html). The `global` section contains direct model initialisation properties.

We will load the default dflowfm build 1D2D yaml file for inspection:

In [ ]:
fn_yml = "dflowfm_build.yml"
with open(fn_yml, "r") as f:
    txt = f.read()
print(txt)

Looking at this file, you see that we will prepare the following information for our model:

- [global](https://deltares.github.io/hydromt_delft3dfm/latest/_generated/hydromt_delft3dfm.DFlowFMModel.html#hydromt_delft3dfm.DFlowFMModel): Our model will be defined in the projected CRS WGS84 EPSG 3857. TO ensure the 1D network is connected, we will allow snapping of 25 meters using argument network_snap_offset. The 1D computational grid for the open water system will be created at 40 meters distance using argument openwater_computation_node_distance. 

- [setup_rivers_from_dem](https://deltares.github.io/hydromt_delft3dfm/latest/_generated/hydromt_delft3dfm.DFlowFMModel.setup_rivers_from_dem.html): We will derive the 1D rivers lines based on the *MERIT Hydro DEM* and that intersects with the region bounding box [12.4331, 46.4661, 12.5212, 46.5369]. River roughness will be a Manning constant of 25.0 and we will use data from *rivers_lin2019* database to prepare rectangular cross-sections. 
- [setup_pipes](https://deltares.github.io/hydromt_delft3dfm/latest/_generated/hydromt_delft3dfm.DFlowFMModel.setup_pipes.html): We will add pipes by assuming that they are located under the roads, with a standard length of 50 meters. The roads are defined in the *grip_roads* data and the standard length are defined in the argument spacing. Roughness will be a WhiteColeBrook constant of 0.003 and we will use a circular cross-sections of 0.5m. The pipes invert levels will be derived from the *MERIT Hydro DEM* assuming a constant depth of the pipe underground of 2 meters. 
- [setup_manholes](https://deltares.github.io/hydromt_delft3dfm/latest/_generated/hydromt_delft3dfm.DFlowFMModel.setup_manholes.html): For a pipe network, manholes are present at pipe connections. Here we do not have specific input data locations for our pipes, so manholes will be generated based on a set of standards specified in manhole_defaults_fn.
- [setup_1dboundary](https://deltares.github.io/hydromt_delft3dfm/latest/_generated/hydromt_delft3dfm.DFlowFMModel.setup_1dboundary.html): Finally as a default, we will apply both upstream and downstream boundaries of our rivers with a constant waterlevel of -2.0 meter a.s.l

- [setup_mesh2d](https://deltares.github.io/hydromt_delft3dfm/latest/_generated/hydromt_delft3dfm.DFlowFMModel.setup_mesh2d.html): we will prepare a regular grid for the region bounding box [12.4331, 46.4661, 12.5212, 46.5369] of *500***500* meters resolution.
- [setup_maps_from_rasterdataset](https://deltares.github.io/hydromt_delft3dfm/latest/_generated/hydromt_delft3dfm.DFlowFMModel.setup_maps_from_rasterdataset.html): prepares gridded elevation (bedlevel) from *merit_hydro*. Here elevation is extracted from *merit_hydro* and any no data is filled using *nearest* method. When running, the Delft3D FM kernel will interpolate it to the mesh 2D grid using the *nearestNb* method.
- [setup_maps_from_raster_reclass](https://deltares.github.io/hydromt_delft3dfm/latest/_generated/hydromt_delft3dfm.DFlowFMModel.setup_maps_from_raster_reclass.html): prepares gridded Manning roughness and infiltration capacity data from landuse, by reclassifying the landuse categories in *vito* landuse data, using parameters values defined per landuse category in *vito_mapping* table. When running, the Delft3D FM kernel will interpolate it to the mesh 2D grid using the *triangulation* method.
- [setup_rainfall_from_constant](https://deltares.github.io/hydromt_delft3dfm/latest/_generated/hydromt_delft3dfm.DFlowFMModel.setup_rainfall_from_constant.html): prepares a constant rainfall rate of *150* mm/day as meteo forcing to the model. 
Feel free to use the links of each components to know more about how each function works and what are all the available settings.
Additionnally to a 1D or 2D models only, we have now added a new method:

- [setup_link1d2d](https://deltares.github.io/hydromt_delft3dfm/latest/_generated/hydromt_delft3dfm.DFlowFMModel.setup_link1d2d.html): we will link our 1D and 2D mesh by setting up links from the 1D network to the 2D grid. As you can imagine, this function can only be run once both the schematisation of the 1D and the 2D grids are finalized. If any change is made to the 1D or the 2D grid, this funcion should be run again.

Feel free to use the links of each components to know more about how each function works and what are all the available settings.

> **NOTE**: 
  In this example, we apply region argument in setup_rivers_from_dem, setup_pipes and setup_mesh2d. They are identical in this example but one can alter them to have different region for rivers, for pipes and for 2dmesh. This allows the user to customize their model as much as possible, as the extent of the 1D river can differ from the extent of the 1D urban network or the 2D inundation grid. The region bbox are in WGS 84 as required, but the user should specify the dflowfm model destination CRS in the `global` section in order to allow to use grid definition data (rivers, pipes) from different data sources or CRS. 

### Building a 1D2D Delft3D FM model

In [ ]:
# NOTE: copy this line (without !) to your shell for more direct feedback - should take about 1 minute or 2 to run
!hydromt build dflowfm "./build/dflowfm_1d2d" -i dflowfm_build.yml -d artifact_data --fo -vv

The example above means the following: run **hydromt build** with:

* `dflowfm` : i.e. build a FM model
* `./build/dflowfm_1d2d` : output model folder
* `-i dflowfm_build.yml`: model build configuration file
* `-d artifact_data`: data catalog to use. Here the default artifact data catalog with global data for Northern Italy.
* `--fo`: force overwritting in case a model already exists in the output folder.
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug and info messages are provided rather than only warnings and errors.

> NOTE: You can use as many data catalogs as you need within the same command line, by repeating the `-d` options and name of the catalog to use several times.

Next we check which files have been created.

In [ ]:
import os

root = "./build/dflowfm_1d2d/"
for path, _, files in os.walk(root):
    print(path)
    for name in files:
        print(f" - {name}")

The model root should contain three files and four folders:

- dimr_config.xml: Configuration file to execute Delft3D FM in Deltares Integrated Model Runner (DIMR).
- hydromt.log: log saved by HydroMT on model building steps.
- hydromt_data.yml: information saved by HydroMT on used datasets for reproducibility.
- **dflowfm** folder: dflowfm files, here for our 2D model. Contains all necessary information to run a 2D model including the mesh2d, and settings and paths to the mesh2d data files in the maps folder. The configuration MDU file is also created.
- **geoms** folder: folder containing a copy of the dflowfm data in an easily readable GIS format rather than the ini or UGRID netcdf file.
- **maps** folder: folder containing regular gridded data that Delft3D FM can interpolate to the 2D mesh grid. Here we see our elevation or bedlevel data, as well as the roughness / friction and infiltration capacity maps.

### Visualize and/or inspect model schematization

* We can now load the **dimr_config.xml** into **Delft3D FM GUI**.
* The **dflowfm plot** [example notebook](https://deltares.github.io/hydromt_delft3dfm/latest/_examples/plot_dflowfm_mesh.html) contains scripts to visualize your model in python: 